# Testing GAN generated data augmentation, using different proportions of data for training GAN 
## This was 

Dataset is COVID_QU_Ex dataset from Kaggle

# Libraries

In [1]:
import os 
import torch
from torch.utils.data import DataLoader
import torchvision
import numpy as np
import random
from PIL import Image
import pickle
import subprocess
import time
import neptune 
import matplotlib
from sklearn.metrics  import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, balanced_accuracy_score, recall_score, precision_score, f1_score
import gc
#This code needs a little bit rework, so testing will be easier

In [2]:
ROOT_DIR = 'Lung_Segmentation_Data'
#run = neptune.init_run()

torch.set_num_threads(12)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
gan_directories = {
    'Test_orig_0.8' : {'dir':'2023-03-14_19-00-08','best':'127.0.0.1-5002'},
    'Test_orig_0.6' : {'dir':'2023-03-15_09-51-22','best':'127.0.0.1-5001'},
    'Test_orig_0.4' : {'dir':'2023-03-16_17-23-13','best':'127.0.0.1-5002'},
    'Test_orig_0.2' : {'dir':'2023-03-17_11-28-34','best':'127.0.0.1-5002'},
    
    'Test_0_0.8' : {'dir':'2023-03-23_15-19-22','best':'127.0.0.1-5002'},
    'Test_0_0.6' : {'dir':'2023-03-24_16-23-49','best':'127.0.0.1-5001'},
    'Test_0_0.4' : {'dir':'2023-03-25_09-50-59','best':'127.0.0.1-5001'},
    'Test_0_0.2' : {'dir':'2023-03-25_16-58-45','best':'127.0.0.1-5003'},

    'Test_1_0.8' : {'dir':'2023-03-21_16-13-32','best':'127.0.0.1-5003'},
    'Test_1_0.6' : {'dir':'2023-03-22_09-56-24','best':'127.0.0.1-5001'},
    'Test_1_0.4' : {'dir':'2023-03-22_18-38-27','best':'127.0.0.1-5000'},
    'Test_1_0.2' : {'dir':'2023-03-23_08-16-51','best':'127.0.0.1-5002'},

    'Test_2_0.8' : {'dir':'2023-03-19_18-20-08','best':'127.0.0.1-5001'},
    'Test_2_0.6' : {'dir':'2023-03-20_08-46-21','best':'127.0.0.1-5001'},
    'Test_2_0.4' : {'dir':'2023-03-20_16-58-27','best':'127.0.0.1-5001'},
    'Test_2_0.2' : {'dir':'2023-03-21_08-49-59','best':'127.0.0.1-5002'},

    'Test_2_nested_0.8' : {'dir':'2023-03-19_18-20-08','best':'127.0.0.1-5001'},
    'Test_2_nested_0.6' : {'dir':'2023-03-20_08-46-21','best':'127.0.0.1-5001'},
    'Test_2_nested_0.4' : {'dir':'2023-03-20_16-58-27','best':'127.0.0.1-5001'},
    'Test_2_nested_0.2' : {'dir':'2023-03-21_08-49-59','best':'127.0.0.1-5002'},

    'Test_3_0.8' : {'dir':'2023-03-17_17-42-29','best':'127.0.0.1-5001'},
    'Test_3_0.6' : {'dir':'2023-03-18_07-56-44','best':'127.0.0.1-5003'},
    'Test_3_0.4' : {'dir':'2023-03-18_23-01-23','best':'127.0.0.1-5002'},
    'Test_3_0.2' : {'dir':'2023-03-19_07-43-42','best':'127.0.0.1-5000'},
     }

In [4]:
# case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_
# {params['augment']}_{params['geoaugment']}_{params['id']}"
# f"{split}_{network_name}_{data_ratio}_{augment}_{geoaugment}"
case_to_neptune_id = {
    "orig_resnet_1_None_False" : "COV1-651",
    "orig_resnet_1_None_True" : "COV1-652",
    
    "orig_resnet_0.8_None_False" : "COV1-60",
    "orig_resnet_0.8_None_True" : "COV1-522",
    "orig_resnet_0.8_oversampling_False" : "COV1-527",
    "orig_resnet_0.8_oversampling_True" : "COV1-528",
    "orig_resnet_0.8_gan_False" : "COV1-273",
    "orig_resnet_0.8_gan_True" : "COV1-274",
    
    "orig_resnet_0.6_None_False" : "COV1-68",
    "orig_resnet_0.6_None_True" : "COV1-264",
    "orig_resnet_0.6_oversampling_False" : "COV1-275",
    "orig_resnet_0.6_oversampling_True" : "COV1-250",
    "orig_resnet_0.6_gan_False" : "COV1-276",
    "orig_resnet_0.6_gan_True" : "COV1-277",
    
    "orig_resnet_0.4_None_False" : "COV1-72",
    "orig_resnet_0.4_None_True" : "COV1-278",
    "orig_resnet_0.4_oversampling_False" : "COV1-125",
    "orig_resnet_0.4_oversampling_True" : "COV1-245",
    "orig_resnet_0.4_gan_False" : "COV1-279",
    "orig_resnet_0.4_gan_True" : "COV1-280",
    
    "orig_resnet_0.2_None_False" : "COV1-86",
    "orig_resnet_0.2_None_True" : "COV1-101",
    "orig_resnet_0.2_oversampling_False" : "COV1-251",
    "orig_resnet_0.2_oversampling_True" : "COV1-281",
    "orig_resnet_0.2_gan_False" : "COV1-269",
    "orig_resnet_0.2_gan_True" : "COV1-282",
    
    ###################### VGG id's
    "orig_vgg_1_None_False" : "COV1-287",
    "orig_vgg_1_None_True" : "COV1-309",
    
    "orig_vgg_0.8_None_False" : "COV1-288",
    "orig_vgg_0.8_None_True" : "COV1-310",
    "orig_vgg_0.8_oversampling_False" : 'COV1-326', #"COV1-527",
    "orig_vgg_0.8_oversampling_True" : "COV1-335",
    "orig_vgg_0.8_gan_False" : "COV1-354",
    "orig_vgg_0.8_gan_True" : "COV1-379",
    
    "orig_vgg_0.6_None_False" : "COV1-290",
    "orig_vgg_0.6_None_True" : "COV1-302",
    "orig_vgg_0.6_oversampling_False" : "COV1-321",
    "orig_vgg_0.6_oversampling_True" : "COV1-336",
    "orig_vgg_0.6_gan_False" : "COV1-368",
    "orig_vgg_0.6_gan_True" : "COV1-391",
    
    "orig_vgg_0.4_None_False" : "COV1-292",
    "orig_vgg_0.4_None_True" : "COV1-314",
    "orig_vgg_0.4_oversampling_False" : "COV1-322",
    "orig_vgg_0.4_oversampling_True" : "COV1-339",
    "orig_vgg_0.4_gan_False" : "COV1-359",
    "orig_vgg_0.4_gan_True" : "COV1-383",
    
    "orig_vgg_0.2_None_False" : "COV1-294",
    "orig_vgg_0.2_None_True" : "COV1-306",
    "orig_vgg_0.2_oversampling_False" : "COV1-325",
    "orig_vgg_0.2_oversampling_True" : "COV1-341",
    "orig_vgg_0.2_gan_False" : "COV1-388",
    "orig_vgg_0.2_gan_True" : "COV1-387",
    
    ###################### efficient id's
    "orig_efficient_1_None_False" : "COV1-400",
    "orig_efficient_1_None_True" : "COV1-423",
    
    "orig_efficient_0.8_None_False" : "COV1-408",
    "orig_efficient_0.8_None_True" : "COV1-427",
    "orig_efficient_0.8_oversampling_False" : 'COV1-440', #"COV1-527",
    "orig_efficient_0.8_oversampling_True" : "COV1-473",
    "orig_efficient_0.8_gan_False" : "COV1-478",
    "orig_efficient_0.8_gan_True" : "COV1-509",
    
    "orig_efficient_0.6_None_False" : "COV1-410",
    "orig_efficient_0.6_None_True" : "COV1-430",
    "orig_efficient_0.6_oversampling_False" : "COV1-454",
    "orig_efficient_0.6_oversampling_True" : "COV1-463",
    "orig_efficient_0.6_gan_False" : "COV1-490",
    "orig_efficient_0.6_gan_True" : "COV1-512",
    
    "orig_efficient_0.4_None_False" : "COV1-414",
    "orig_efficient_0.4_None_True" : "COV1-433",
    "orig_efficient_0.4_oversampling_False" : "COV1-449",
    "orig_efficient_0.4_oversampling_True" : "COV1-468",
    "orig_efficient_0.4_gan_False" : "COV1-493",
    "orig_efficient_0.4_gan_True" : "COV1-514",
    
    "orig_efficient_0.2_None_False" : "COV1-421",
    "orig_efficient_0.2_None_True" : "COV1-436",
    "orig_efficient_0.2_oversampling_False" : "COV1-457",
    "orig_efficient_0.2_oversampling_True" : "COV1-475",
    "orig_efficient_0.2_gan_False" : "COV1-494",
    "orig_efficient_0.2_gan_True" : "COV1-518",
}

# Making datasets

In [5]:

class CustomDataset(torchvision.datasets.ImageFolder):
    """
        A simple imagedatset for storing data
    """
    #Imagefolder for efficiency
    def __init__(self, images, transform):
        target_dir = os.path.join(ROOT_DIR, 'original')
        super().__init__(target_dir, transform)
        self.samples = images
        self.imgs = images

def DatasetMaker(split, mode=None, data_ratio=1, transform = None, geoaugment=False, random_affine = 0, seed = 0):
    """
        Returns a CustomDataset with given parameters
        split: str, determines train-test to use; options: 'orig', '0', '1', '2', '3'
        mode: str, 'oversampling', 'gan' or None
            'oversampling' : oversample with real images to balance classes
                    'gan' : balance datasets with gan generated images (uses data_ratio to figure out which gan to use)
                    None  : dataset won't be balanced
        data_ratio (optional): int, the ratio of the training covid data to be used
                    options:
                        '1' : all data
                        '0.8' : 80% of training images
                        '0.6' : 60% of training images
                        '0.4' : 40% of training images
                        '0.2' : 20% of training images
        transform (optional): torch.Compose instance, sets the dataset's transforms
        geougment (optional): bool, uses basic data augmentation techmiques
        seed (optional): int, seed to use for reproducibility
    """
    #This code prepares my fixed set of images to replace the Imagefolder's original images
    #Maybe this could be done nicer with overwriting the DatasetFolder's find_classes method, but currently this works
    # the idea is to make (route, index) pairs for later loading in images
    classes = ['normal', 'viral', 'covid']

    #Determining the root directories for original images 
    orig_dirs = {
    'normal' : f'{ROOT_DIR}/original/Normal',
    'viral' : f'{ROOT_DIR}/original/Non-Covid',
    'covid' : f'{ROOT_DIR}/original/COVID-19'
    }
    #Determining the root directories for generated images 
    fake_dirs = {
        'gan_0.8' : f'{ROOT_DIR}/generated/Test_{split}/gan_0.8',
        'gan_0.6' : f'{ROOT_DIR}/generated/Test_{split}/gan_0.6',
        'gan_0.4' : f'{ROOT_DIR}/generated/Test_{split}/gan_0.4',
        'gan_0.2' : f'{ROOT_DIR}/generated/Test_{split}/gan_0.2'
    }
    #Determining the root directories for files which contain the names of the COVID19 pictures 
    indicies_files = {
        'gan_0.8' : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_0.8_gan.pkl', 
        'gan_0.6' : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_0.6_gan.pkl',
        'gan_0.4' : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_0.4_gan.pkl',
        'gan_0.2' : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_0.2_gan.pkl',
        'test'  : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_test.pkl',
        'train'  : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_train_and_val.pkl'
    }
    class_idx = {
        'covid': 0, 
        'viral': 1,
        'normal': 2,
        'gan_0.8' : 0,
        'gan_0.6' : 0,
        'gan_0.4' : 0,
        'gan_0.2' : 0
    }
    idx_to_class ={
        0: 'covid',
        1: 'viral',
        2: 'normal'
    }
    valid_ratios = [1, 0.8, 0.6, 0.4, 0.2]

    #The dictionary that will contain the route for the several image classes
    source_dir = {}
    train_images = []

    #creating the sources for the classes
    for class_name in classes: 
        source_dir[class_name] = orig_dirs[class_name]
    
    #Get all training images
    file = indicies_files['train'] 
    imgs = load_images_from_file(file)
    imgs = [*imgs[0],*imgs[1]] #The file contains (train, val) sets
    
    #Making the paths for the training images
    num_of_covid_imgs = 0 #?
    for x in imgs: 
        class_of_x = idx_to_class[x[1]] #The images are saved in (image_name, class_index) format
        item = os.path.join(source_dir[class_of_x],x[0]),class_idx[class_of_x] 
        if item[1] == class_idx['covid']: num_of_covid_imgs +=1 #?
        train_images.append(item)
    #If data_ratio is not 1, we need to change the covid images of the dataset
    if data_ratio!=1:
        num_of_covid_imgs = 0
        train_images = [x for x in train_images if x[1]!=class_idx['covid']]
        file = indicies_files[f'gan_{data_ratio}']
        imgs = load_images_from_file(file)
        for x in imgs:
            class_of_x = idx_to_class[x[1]]
            item = os.path.join(source_dir[class_of_x],x[0]),class_idx[class_of_x] 
            if item[1] == class_idx['covid']: num_of_covid_imgs +=1 
            train_images.append(item)
    
    #If picture generation is needed we want to know the average class size and missing number of covid images
    average_class_size = (len(train_images)-num_of_covid_imgs)//2
    missing_images = max(0, average_class_size - num_of_covid_imgs)

    #Determining wether image generation is needed and which kind of it then making the generation
    if mode=='gan' and (data_ratio in valid_ratios):
        gan = f'gan_{data_ratio}'
        gan_dir = fake_dirs[gan]
        #generate_images_to_dir(split, data_ratio, gan, gan_dir, missing_images) #?
        gan_ims = []
        for x in os.listdir(gan_dir): #optimize further
            if x.lower().endswith('jpg'):
                item = os.path.join(gan_dir, x), class_idx['covid']
                gan_ims.append(item)
        sample = random.sample(gan_ims, missing_images)
        train_images = [*train_images,*sample]
    elif mode=='oversampling':
        covid_images = [x for x in train_images if x[1]==class_idx['covid']]
        batch_size = len(covid_images)
        while batch_size <= missing_images:
            train_images = [*train_images, *covid_images]
            missing_images -= batch_size
        if missing_images>0:
            sample = random.sample(covid_images, missing_images)
            train_images = [*train_images, *sample]
    
    #Making the paths for the test images
    test_images = []
    test_imgs = load_images_from_file(indicies_files['test'])
    for x in test_imgs:
        class_of_x = idx_to_class[x[1]]
        item = os.path.join(source_dir[class_of_x],x[0]),class_idx[class_of_x] #This should be correct
        test_images.append(item)
    random.Random(seed).shuffle(test_images)
    bound = len(test_images)//2
    val_images = test_images[bound:]
    test_images = test_images[:bound]
    #Making some basic transforms 
    if transform is None:
        transforms = [torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    torchvision.transforms.Grayscale(num_output_channels=1)]
    else:
        transforms = transform

    #Checking wether geometric augmentation is needed ( classic augmentation methods)
    if geoaugment:
        augmentation_transforms = [torchvision.transforms.RandomAffine(random_affine),
                            torchvision.transforms.Resize(size=(140,140)),
                                   #torchvision.transforms.RandomCrop(size = (128,128))]
                            torchvision.transforms.FiveCrop(size = (128,128)),
                            torchvision.transforms.Lambda(lambda crops: torch.stack([crop for crop in crops]))]
        transforms = [*transforms,*augmentation_transforms]
    else:
        size_transform = [torchvision.transforms.Resize(size=(128, 128))]
        transforms = [*transforms,*size_transform]
    transforms = torchvision.transforms.Compose(transforms)
    train_dataset = CustomDataset(train_images, transforms)
    val_dataset = CustomDataset(val_images, transforms)
    test_dataset = CustomDataset(test_images,  transforms)
    return train_dataset, val_dataset, test_dataset

def load_images_from_file(file):
    with open(file, 'rb') as file:
        data = pickle.load(file)
    return data   
    
#def generate_images_to_dir(split, data_ratio, gan, directory, size):
#    """
#        Generates pictures with a given gan, to a given directory
#    """
#    #Goes into Lipizzaner's directory and then generates images with a given GAN (this function is a Lipizzaner built-in method)
#    #Then returns into this directory
#    
#    curr_dir = os.getcwd()
#    print(curr_dir)
#    
#    lippi_dir = '/home/bbernard/lipizzaner-covidgan-master/src/'  #Change this on server
#    
#    output_dir = os.path.join(curr_dir, directory) #?
#   
#   #Gan to use is determined by the split and the data_ratio parameters
#    gan_dir = gan_directories[f'Test_{split}_{data_ratio}'] 
#    src_dir = os.path.join(lippi_dir, f'output/lipizzaner_gan/master/{gan_dir}/127.0.0.1-5000')
#    
#    config_file = os.path.join(lippi_dir, f'configuration/covid-qu-conv/Test_{split}/covidqu_{data_ratio}.yml')
#    
#    #man = os.path.join(lippi_dir, 'main.py')#
#
#    code =f'python main.py generate --mixture-source {src_dir} -o {output_dir} --sample-size {size} -f {config_file}'
#    os.chdir(lippi_dir)
#    subprocess.run(code, shell=True)
#    os.chdir(curr_dir)

## Models


In [6]:
def get_model(name):
    """
        Returns pretrained models
    """
    if name=="resnet":
        resnet18 = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        resnet18.conv1= torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet18.fc = torch.nn.Linear(in_features=512, out_features=3)
        resnet18.get_name = 'resnet18'
        return resnet18
    elif name=="vgg":
        vgg16 = torchvision.models.vgg16(weights = torchvision.models.VGG16_Weights.IMAGENET1K_V1)
        vgg16.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        vgg16.classifier[6] = torch.nn.Linear(in_features=4096, out_features=3, bias=True)
        vgg16.get_name = 'vgg16'
        return vgg16
    elif name=="efficient":
        efficient = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        efficient.features[0][0] = torch.nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        efficient.classifier[1] = torch.nn.Linear(in_features=1280, out_features=3, bias=True)
        efficient.get_name = 'efficientnet_b0'
        return efficient
    else:
        print("Not implemented")

## Train function

In [7]:
def train(epochs, net, loss_fn, optimizer, weight_decay, train_dataset, test_dataset, batch_size, shuffle, neptune_run): #, neptune_run ):
    """
        A simple train function 
        Params: 
            epoch: number of epochs to train for
            modell: The neural network to train
            loss_fn: Loss function instance
            optimizer: optimizer instance
            train_dataset (Dataset)
            test_dataset (Dataset)
            batch_size (int)
            shuffle (bool) 
    """
    train_dl = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers=0, shuffle= shuffle)
    test_dl =  torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, num_workers=0, shuffle= shuffle)
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=weight_decay)
    print('Starting training..')
    net.to(device)
    for e in range(epochs):
        print('='*20)
        print(f'Starting epoch {e + 1}/{epochs}')
        print('='*20)

        train_iter = iter(train_dl)
        
        train_accuracy = 0.
        train_loss = 0.
        
        sample_num = 0 

        net.train() #set model to training phase
        
        #Training 
        batch_num = 0
        
        while batch_num< len(train_dl):
            images, labels = next(train_iter)
            if len(images.size())==5:
                bs, ncrops, c, h, w = images.size()
            else:
                bs, c, h, w = images.size()
                ncrops = 1
            images = images.view(-1, c,h,w)
            images = images.to(device) 
            
            labels = labels.to(device)
            #labels = labels.repeat_interleave(ncrops)
            
            outputs = net(images)
            outputs = outputs.view(bs, ncrops, -1).mean(1) 
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #train_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            
            sample_num += len(labels)
            train_accuracy += sum((preds == labels).cpu().numpy())
            train_loss += loss.item()
            batch_num += 1
        
        train_loss/= len(train_dl)
        train_accuracy/= len(train_dataset)
        print(f'Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}')
        neptune_run['train/train_loss'].append(train_loss)
        neptune_run['train/train_accuracy'].append(train_accuracy)

        val_loss = 0.
        val_accuracy = 0.

        val_batch_num = 0
        val_iter = iter(test_dl)
        y_true = []
        y_pred = []

        net.eval()
        while val_batch_num < len(val_iter):
            images, labels = next(val_iter)
            if len(images.size())==5:
                bs, ncrops, c, h, w = images.size()
            else:
                bs, c, h, w = images.size()
                ncrops = 1
            
            images = images.view(-1, c,h,w)
            images = images.to(device) 
            
            #labels = labels.repeat_interleave(ncrops)
            y_true.extend(labels)
            labels = labels.to(device)
            
            outputs = net(images)
            outputs = outputs.view(bs, ncrops, -1).mean(1) 
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.detach().cpu())
            val_accuracy += sum((preds == labels).cpu().numpy())
            val_batch_num += 1

        val_loss /= len(test_dl)
        val_accuracy = val_accuracy/len(test_dataset)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')
        scheduler.step()
        net.train()

        neptune_run['train/val_loss'].append(val_loss)
        neptune_run['train/val_accuracy'].append(val_accuracy)

        curr_conf_matrix = confusion_matrix(y_true, y_pred) 
        curr_conf_matrix = curr_conf_matrix / np.sum(curr_conf_matrix)
        im = ConfusionMatrixDisplay(curr_conf_matrix, display_labels=["covid", "viral","normal"]).plot()
        neptune_run['metrics/conf_matrix'].append(im.figure_) #, description=f"Confusion matrix in the iteration: {iteration}"  File.as_image(curr_conf_matrix))
        matplotlib.pyplot.close()
        neptune_run['metrics/acc'].append( accuracy_score(y_true, y_pred))
        neptune_run['metrics/bal_acc'].append(balanced_accuracy_score(y_true, y_pred))
        neptune_run['metrics/recall'].append(recall_score(y_true, y_pred,average ="micro"))
        neptune_run['metrics/precision'].append( precision_score(y_true, y_pred, average ="micro"))
        neptune_run['metrics/f1'].append( f1_score(y_true, y_pred,average ="micro"))    
        #neptune_run['val/loss'].append(val_loss)
        #neptune_run['val/accuracy'].append(val_accuracy)
        torch.cuda.empty_cache() 
        
    print('Training complete..')
    return net


In [8]:
def train_and_test(epochs, net, loss_fn, optimizer, weight_decay, train_dataset, test_dataset, batch_size, shuffle, neptune_run): #, neptune_run ):
    """
        A simple train function 
        Params: 
            epoch: number of epochs to train for
            modell: The neural network to train
            loss_fn: Loss function instance
            optimizer: optimizer instance
            train_dataset (Dataset)
            test_dataset (Dataset)
            batch_size (int)
            shuffle (bool) 
    """
    train_dl = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers=0, shuffle= shuffle)
    test_dl =  torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, num_workers=0, shuffle= shuffle)
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=weight_decay)
    print('Starting training..')
    net.to(device)
    for e in range(epochs):
        print('='*20)
        print(f'Starting epoch {e + 1}/{epochs}')
        print('='*20)

        train_iter = iter(train_dl)
        
        train_accuracy = 0.
        train_loss = 0.
        
        sample_num = 0 

        net.train() #set model to training phase
        
        #Training 
        batch_num = 0
        
        while batch_num< len(train_dl):
            images, labels = next(train_iter)
            if len(images.size())==5:
                bs, ncrops, c, h, w = images.size()
            else:
                bs, c, h, w = images.size()
                ncrops = 1
            images = images.view(-1, c,h,w)
            images = images.to(device) 
            
            labels = labels.to(device)
            
            outputs = net(images)
            outputs = outputs.view(bs, ncrops, -1).mean(1) 
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            
            sample_num += len(labels)
            train_accuracy += sum((preds == labels).cpu().numpy())
            train_loss += loss.item()
            batch_num += 1
        
        train_loss/= len(train_dl)
        train_accuracy/= len(train_dataset)
        print(f'Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}')
        neptune_run['train/train_loss'].append(train_loss)
        neptune_run['train/train_accuracy'].append(train_accuracy)
        scheduler.step()

    val_loss = 0.
    val_accuracy = 0.

    val_batch_num = 0
    val_iter = iter(test_dl)
    y_true = []
    y_pred = []

    net.eval()
    while val_batch_num < len(val_iter):
        images, labels = next(val_iter)
        if len(images.size())==5:
            bs, ncrops, c, h, w = images.size()
        else:
            bs, c, h, w = images.size()
            ncrops = 1

        images = images.view(-1, c,h,w)
        images = images.to(device) 

        #labels = labels.repeat_interleave(ncrops)
        y_true.extend(labels)
        labels = labels.to(device)

        outputs = net(images)
        outputs = outputs.view(bs, ncrops, -1).mean(1) 
        loss = loss_fn(outputs, labels)
        val_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.detach().cpu())
        val_accuracy += sum((preds == labels).cpu().numpy())
        val_batch_num += 1

    val_loss /= len(test_dl)
    val_accuracy = val_accuracy/len(test_dataset)
    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')
    net.train()

    neptune_run['test/test_loss'].append(val_loss)
    neptune_run['test/test_accuracy'].append(val_accuracy)

    curr_conf_matrix = confusion_matrix(y_true, y_pred) 
    curr_conf_matrix = curr_conf_matrix / np.sum(curr_conf_matrix)
    im = ConfusionMatrixDisplay(curr_conf_matrix, display_labels=["covid", "viral","normal"]).plot()
    neptune_run['test/metrics/conf_matrix'].append(im.figure_) #, description=f"Confusion matrix in the iteration: {iteration}"  File.as_image(curr_conf_matrix))
    matplotlib.pyplot.close()
    neptune_run['test/metrics/acc'].append( accuracy_score(y_true, y_pred))
    neptune_run['test/metrics/bal_acc'].append(balanced_accuracy_score(y_true, y_pred))
    neptune_run['test/metrics/recall'].append(recall_score(y_true, y_pred,average ="micro"))
    neptune_run['test/metrics/precision'].append( precision_score(y_true, y_pred, average ="micro"))
    neptune_run['test/metrics/f1'].append( f1_score(y_true, y_pred,average ="micro"))    
    #neptune_run['val/loss'].append(val_loss)
    #neptune_run['val/accuracy'].append(val_accuracy)
    torch.cuda.empty_cache() 
        
    print('Training complete..')
    return net


In [9]:
def load_in_network(network_name, case):
    network = get_model(network_name)
    path_string = f"Histories/{network_name}/{case}.pt" 
    network.load_state_dict(torch.load(path_string, map_location='cuda:0'))
    return network

def test_run(split, data_ratio, augment, geoaugment, network_name):
    neptune_id = case_to_neptune_id[f"{split}_{network_name}_{data_ratio}_{augment}_{geoaugment}"]
    neptune_run = neptune.init_run(with_id = neptune_id)
        
    case_string = f"{network_name}_{split}_{data_ratio}_{augment}_{geoaugment}_{neptune_id}"
    network = load_in_network(network_name, case_string)
    network.to(device)
    print("Succesfully loaded network")
    degree = neptune_run["params/degree"].fetch()
    _,_,test_dataset = DatasetMaker(split = split,mode = augment, 
                                data_ratio = data_ratio, geoaugment = geoaugment, random_affine = degree)
    loss_fn = torch.nn.CrossEntropyLoss()
    batch_size = 128 #for now
    
    test_dl =  torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, num_workers=0)
    test_iter = iter(test_dl)
    
    y_true = []
    y_pred = []
    test_accuracy = 0.0
    test_loss = 0.0
    test_batch_num = 0
    
    network.eval()
    print("Starting testing")
    while test_batch_num < len(test_iter):
        images, labels = next(test_iter)
        if len(images.size())==5:
            bs, ncrops, c, h, w = images.size()
        else:
            bs, c, h, w = images.size()
            ncrops = 1

        images = images.view(-1, c,h,w)
        images = images.to(device) 

        #labels = labels.repeat_interleave(ncrops)
        y_true.extend(labels)
        labels = labels.to(device)

        outputs = network(images)
        outputs = outputs.view(bs, ncrops, -1).mean(1) 
        loss = loss_fn(outputs, labels)
        test_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.detach().cpu())
        test_accuracy += sum((preds == labels).cpu().numpy())
        test_batch_num += 1

    test_loss /= len(test_dl)
    test_accuracy = test_accuracy/len(test_dataset)
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}')

    neptune_run['test/test_loss'].append(test_loss)
    neptune_run['test/test_accuracy'].append(test_accuracy)

    curr_conf_matrix = confusion_matrix(y_true, y_pred) 
    curr_conf_matrix = curr_conf_matrix / np.sum(curr_conf_matrix)
    im = ConfusionMatrixDisplay(curr_conf_matrix, display_labels=["covid", "viral","normal"]).plot()
    neptune_run['test/metrics/conf_matrix'].append(im.figure_) #, description=f"Confusion matrix in the iteration: {iteration}"  File.as_image(curr_conf_matrix))
    matplotlib.pyplot.close()
    neptune_run['test/metrics/acc'].append( accuracy_score(y_true, y_pred))
    neptune_run['test/metrics/bal_acc'].append(balanced_accuracy_score(y_true, y_pred))
    neptune_run['test/metrics//recall'].append(recall_score(y_true, y_pred,average ="micro"))
    neptune_run['test/metrics/precision'].append( precision_score(y_true, y_pred, average ="micro"))
    neptune_run['test/metrics/f1'].append( f1_score(y_true, y_pred,average ="micro"))    
    torch.cuda.empty_cache() 
    neptune_run.stop()
    
#Maybe save the individual test results?
#Somehow the data needs to be collectable

In [10]:
def save_history(network,folder, case_string): #,history
    """
        A simple function that saves the histories and the clasificator net
    """
    FILEBASE = f"Histories/{folder}/{case_string}"
    torch.save(network.state_dict(), FILEBASE + '.pt')
    #with open(FILEBASE + '-history.pkl', 'wb') as file:
    #    pickle.dump(history, file)
    #    print(f'{FILEBASE} instance saved')


params = {
'split' : 'orig',
'data_ratio' : 0.8,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'oversampling',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "efficient", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

In [11]:
def collect_params(split, data_ratio, augment, geoaugment, network_name):
    neptune_id = case_to_neptune_id[f"orig_{network_name}_{data_ratio}_{augment}_{geoaugment}"]
    neptune_run = neptune.init_run(with_id = neptune_id)
    
    try:
        degree = neptune_run["params/degree"].fetch()
    except Exception:
        degree = 4
        
    try: 
        adam_weight_decay = neptune_run["params/adam_weight_decay"].fetch()
    except Exception:
        adam_weight_decay = 1
        
    try:
        weight_decay = neptune_run["params/weight_decay"].fetch()
    except Exception:
        weight_decay = 1
    
    params = {
    'split' : split,
    'data_ratio' : data_ratio,   #1, 0.8, 0.6, 0.4, 0.2
    'augment' : augment,       #gan, oversampling, None
    'geoaugment': geoaugment,
    'network_name' : network_name, #resnet, vgg, efficient
    'id': time.strftime("%H-%M-%S"),
    'loss_function' : "CrossEntropyLoss",
    'optimizer' : neptune_run["params/optimizer"].fetch(),
    'batch_size' : neptune_run["params/batch_size"].fetch(),
    'epochs' : 30, #30/40
    'adam_lr' : neptune_run["params/adam_lr"].fetch(),
    'adam_weight_decay' : adam_weight_decay,
    'weight_decay':weight_decay,
    'degree': degree
    }
    neptune_run.stop()
    return params 

def train_and_test_run(split, data_ratio, augment, geoaugment, network_name):
    params = collect_params(split, data_ratio, augment, geoaugment, network_name)
    
    network = get_model(params['network_name'])
    if params['optimizer']=="adam":
        optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
    elif params['optimizer']=="adamW":
        optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
    loss_fn = torch.nn.CrossEntropyLoss()
    
    train_dataset, _ ,val_dataset = DatasetMaker(split = params['split'],mode = params['augment'], 
                                    data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

    neptune_run = neptune.init_run() 
    run_id = neptune_run["sys/id"].fetch()
    neptune_run['algorithm'] = "Classifier"
    neptune_run['params'] = params
    
    case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
    print("Currently working on: ", case_string)
    network = train_and_test(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                    train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
    neptune_run.stop()
    torch.cuda.empty_cache()
    
    save_history(network, params['network_name'], case_string)
    
#Maybe save the individual test results?
#Somehow the data needs to be collectable

#### Testing classificators

In [29]:
train_and_test_run("0",1,"None","False","resnet")

https://app.neptune.ai/aielte/CovidGAN/e/COV1-80
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-80/metadata
https://app.neptune.ai/aielte/CovidGAN/e/COV1-536
Currently working on:  resnet_0_1_None_False_COV1-536
Starting training..
Starting epoch 1/30
Training Loss: 0.3724, Accuracy: 0.8528
Validation Loss: 0.2199, Accuracy: 0.9166
Starting epoch 2/30
Training Loss: 0.2442, Accuracy: 0.9053
Validation Loss: 0.1898, Accuracy: 0.9349
Starting epoch 3/30
Training Loss: 0.1991, Accuracy: 0.9220
Validation Loss: 0.1995, Accuracy: 0.9255
Starting epoch 4/30
Training Loss: 0.1751, Accuracy: 0.9319
Validation Loss: 0.1566, Accuracy: 0.9502
Starting epoch 5/30
Training Loss: 0.1502, Accuracy: 0.9411
Validation Loss: 0.1510, Accuracy: 0.9476
Starting epoch 6/30
Training Loss: 0.1331, Accuracy: 0.9475
Validation Loss: 0.1764, Accuracy: 0.9408
Starting e

In [ ]:
split = "6_nested"
networks = ["resnet", "vgg", "efficient"]
ratios = [1, 0.8, 0.6, 0.4, 0.2]
augments = ["None", "oversampling", "gan"]
geoaugments = [False, True]

for network in networks:
    for ratio in ratios:
        if ratio!=1:
            for augment in augments:
                for geo in geoaugments:
                    train_and_test_run(split,ratio,augment,geo,network)
        else:
            for geo in geoaugments:
                train_and_test_run(split,ratio,"None",geo,network)
            


/tmp/ipykernel_2786730/448869524.py:3: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  neptune_run = neptune.init_run(with_id = neptune_id)


https://app.neptune.ai/aielte/CovidGAN/e/COV1-651
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-651/metadata
https://app.neptune.ai/aielte/CovidGAN/e/COV1-1006
Currently working on:  resnet_6_nested_1_None_False_COV1-1006
Starting training..
Starting epoch 1/30


In [ ]:
split = "7_nested"
networks = ["vgg", "efficient"]
ratios = [1, 0.8, 0.6, 0.4, 0.2]
augments = ["None", "oversampling", "gan"]
geoaugments = [False, True]

for network in networks:
    for ratio in ratios:
        if ratio!=1:
            for augment in augments:
                for geo in geoaugments:
                    train_and_test_run(split,ratio,augment,geo,network)
        else:
            for geo in geoaugments:
                train_and_test_run(split,ratio,"None",geo,network)
            

https://app.neptune.ai/aielte/CovidGAN/e/COV1-287
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-287/metadata
https://app.neptune.ai/aielte/CovidGAN/e/COV1-1055
Currently working on:  vgg_7_nested_1_None_False_COV1-1055
Starting training..
Starting epoch 1/30
Training Loss: 0.4294, Accuracy: 0.8248
Starting epoch 2/30


In [ ]:
train_and_test_run("6_nested",0.2,"gan",False,"resnet")

In [ ]:
train_and_test_run("7_nested",0.2,"gan",False,"resnet")

/tmp/ipykernel_318659/448869524.py:3: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  neptune_run = neptune.init_run(with_id = neptune_id)


https://app.neptune.ai/aielte/CovidGAN/e/COV1-269
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-269/metadata
https://app.neptune.ai/aielte/CovidGAN/e/COV1-1162
Currently working on:  resnet_7_nested_0.2_gan_False_COV1-1162
Starting training..
Starting epoch 1/30
Training Loss: 0.3122, Accuracy: 0.8749
Starting epoch 2/30
Training Loss: 0.2021, Accuracy: 0.9211
Starting epoch 3/30
Training Loss: 0.1558, Accuracy: 0.9389
Starting epoch 4/30
Training Loss: 0.1220, Accuracy: 0.9533
Starting epoch 5/30
Training Loss: 0.0988, Accuracy: 0.9625
Starting epoch 6/30
Training Loss: 0.0418, Accuracy: 0.9854
Starting epoch 7/30
Training Loss: 0.0201, Accuracy: 0.9938
Starting epoch 8/30
Training Loss: 0.0107, Accuracy: 0.9970
Starting epoch 9/30
Training Loss: 0.0060, Accuracy: 0.9987
Starting epoch 10/30
Training Loss: 0.0037, Accuracy: 0.9992
Starting 

In [13]:
split = "1"
networks = ["resnet","vgg","efficient"]
ratios = [1, 0.8, 0.6, 0.4, 0.2]
augments = ["None", "oversampling", "gan"]
geoaugments = [False, True]

for network in networks:
    for ratio in ratios:
        if ratio!=1:
            for augment in augments:
                if network=="efficient":
                    for geo in geoaugments:
                        train_and_test_run(split,ratio,augment,geo,network)
                else:
                    train_and_test_run(split,ratio,augment,False,network)
        else:
            if network == "efficient":
                for geo in geoaugments:
                    train_and_test_run(split,ratio,"None",geo,network)
            else:
                train_and_test_run(split,ratio,"None",False,network)

https://app.neptune.ai/aielte/CovidGAN/e/COV1-80
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-80/metadata
https://app.neptune.ai/aielte/CovidGAN/e/COV1-589
Currently working on:  resnet_0_1_None_False_COV1-589
Starting training..
Starting epoch 1/30
Training Loss: 0.3852, Accuracy: 0.8489
Starting epoch 2/30
Training Loss: 0.2269, Accuracy: 0.9120
Starting epoch 3/30
Training Loss: 0.1705, Accuracy: 0.9336
Starting epoch 4/30
Training Loss: 0.1335, Accuracy: 0.9477
Starting epoch 5/30
Training Loss: 0.0992, Accuracy: 0.9632
Starting epoch 6/30
Training Loss: 0.0755, Accuracy: 0.9717
Starting epoch 7/30
Training Loss: 0.0598, Accuracy: 0.9776
Starting epoch 8/30
Training Loss: 0.0528, Accuracy: 0.9815
Starting epoch 9/30
Training Loss: 0.0376, Accuracy: 0.9864
Starting epoch 10/30
Training Loss: 0.0392, Accuracy: 0.9856
Starting epoch 11/30


Training Loss: 0.2694, Accuracy: 0.8959
Starting epoch 3/30
Training Loss: 0.2175, Accuracy: 0.9157
Starting epoch 4/30
Training Loss: 0.1843, Accuracy: 0.9297
Starting epoch 5/30
Training Loss: 0.1557, Accuracy: 0.9405
Starting epoch 6/30
Training Loss: 0.1248, Accuracy: 0.9547
Starting epoch 7/30
Training Loss: 0.1046, Accuracy: 0.9602
Starting epoch 8/30
Training Loss: 0.0853, Accuracy: 0.9679
Starting epoch 9/30
Training Loss: 0.0693, Accuracy: 0.9744
Starting epoch 10/30
Training Loss: 0.0587, Accuracy: 0.9788
Starting epoch 11/30
Training Loss: 0.0486, Accuracy: 0.9825
Starting epoch 12/30
Training Loss: 0.0489, Accuracy: 0.9826
Starting epoch 13/30
Training Loss: 0.0423, Accuracy: 0.9845
Starting epoch 14/30
Training Loss: 0.0383, Accuracy: 0.9856
Starting epoch 15/30
Training Loss: 0.0375, Accuracy: 0.9867
Starting epoch 16/30
Training Loss: 0.0326, Accuracy: 0.9889
Starting epoch 17/30
Training Loss: 0.0354, Accuracy: 0.9879
Starting epoch 18/30
Training Loss: 0.0323, Accuracy

Training Loss: 0.0919, Accuracy: 0.9667
Starting epoch 9/30
Training Loss: 0.0728, Accuracy: 0.9716
Starting epoch 10/30
Training Loss: 0.0602, Accuracy: 0.9780
Starting epoch 11/30
Training Loss: 0.0506, Accuracy: 0.9814
Starting epoch 12/30
Training Loss: 0.0429, Accuracy: 0.9844
Starting epoch 13/30
Training Loss: 0.0418, Accuracy: 0.9843
Starting epoch 14/30
Training Loss: 0.0347, Accuracy: 0.9870
Starting epoch 15/30
Training Loss: 0.0323, Accuracy: 0.9883
Starting epoch 16/30
Training Loss: 0.0337, Accuracy: 0.9880
Starting epoch 17/30
Training Loss: 0.0302, Accuracy: 0.9894
Starting epoch 18/30
Training Loss: 0.0252, Accuracy: 0.9907
Starting epoch 19/30
Training Loss: 0.0261, Accuracy: 0.9911
Starting epoch 20/30
Training Loss: 0.0278, Accuracy: 0.9910
Starting epoch 21/30
Training Loss: 0.0227, Accuracy: 0.9924
Starting epoch 22/30
Training Loss: 0.0247, Accuracy: 0.9913
Starting epoch 23/30
Training Loss: 0.0169, Accuracy: 0.9946
Starting epoch 24/30
Training Loss: 0.0181, Ac

Training Loss: 0.0300, Accuracy: 0.9886
Starting epoch 15/30
Training Loss: 0.0268, Accuracy: 0.9898
Starting epoch 16/30
Training Loss: 0.0296, Accuracy: 0.9900
Starting epoch 17/30
Training Loss: 0.0226, Accuracy: 0.9926
Starting epoch 18/30
Training Loss: 0.0255, Accuracy: 0.9915
Starting epoch 19/30
Training Loss: 0.0230, Accuracy: 0.9919
Starting epoch 20/30
Training Loss: 0.0210, Accuracy: 0.9926
Starting epoch 21/30
Training Loss: 0.0210, Accuracy: 0.9925
Starting epoch 22/30
Training Loss: 0.0195, Accuracy: 0.9934
Starting epoch 23/30
Training Loss: 0.0178, Accuracy: 0.9946
Starting epoch 24/30
Training Loss: 0.0150, Accuracy: 0.9949
Starting epoch 25/30
Training Loss: 0.0162, Accuracy: 0.9940
Starting epoch 26/30
Training Loss: 0.0184, Accuracy: 0.9937
Starting epoch 27/30
Training Loss: 0.0163, Accuracy: 0.9943
Starting epoch 28/30
Training Loss: 0.0151, Accuracy: 0.9946
Starting epoch 29/30
Training Loss: 0.0133, Accuracy: 0.9957
Starting epoch 30/30
Training Loss: 0.0121, A

Training Loss: 0.0167, Accuracy: 0.9943
Starting epoch 21/30
Training Loss: 0.0174, Accuracy: 0.9939
Starting epoch 22/30
Training Loss: 0.0238, Accuracy: 0.9923
Starting epoch 23/30
Training Loss: 0.0136, Accuracy: 0.9949
Starting epoch 24/30
Training Loss: 0.0148, Accuracy: 0.9948
Starting epoch 25/30
Training Loss: 0.0147, Accuracy: 0.9949
Starting epoch 26/30
Training Loss: 0.0131, Accuracy: 0.9959
Starting epoch 27/30
Training Loss: 0.0171, Accuracy: 0.9941
Starting epoch 28/30
Training Loss: 0.0129, Accuracy: 0.9952
Starting epoch 29/30
Training Loss: 0.0105, Accuracy: 0.9963
Starting epoch 30/30
Training Loss: 0.0130, Accuracy: 0.9955
Validation Loss: 0.2864, Accuracy: 0.9352
Training complete..
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
All 8 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-5

...

Training Loss: 0.3990, Accuracy: 0.8417
Starting epoch 2/30
Training Loss: 0.2582, Accuracy: 0.9017
Starting epoch 3/30
Training Loss: 0.2058, Accuracy: 0.9220
Starting epoch 4/30
Training Loss: 0.1908, Accuracy: 0.9272
Starting epoch 5/30
Training Loss: 0.1618, Accuracy: 0.9382
Starting epoch 6/30
Training Loss: 0.1473, Accuracy: 0.9435
Starting epoch 7/30
Training Loss: 0.1328, Accuracy: 0.9497
Starting epoch 8/30
Training Loss: 0.1187, Accuracy: 0.9555
Starting epoch 9/30
Training Loss: 0.1085, Accuracy: 0.9583
Starting epoch 10/30
Training Loss: 0.0986, Accuracy: 0.9605
Starting epoch 11/30
Training Loss: 0.0943, Accuracy: 0.9629
Starting epoch 12/30
Training Loss: 0.0806, Accuracy: 0.9687
Starting epoch 13/30
Training Loss: 0.0761, Accuracy: 0.9728
Starting epoch 14/30
Training Loss: 0.0757, Accuracy: 0.9714
Starting epoch 15/30
Training Loss: 0.0689, Accuracy: 0.9738
Starting epoch 16/30
Training Loss: 0.0665, Accuracy: 0.9748
Starting epoch 17/30
Training Loss: 0.0649, Accuracy:

Training Loss: 0.1313, Accuracy: 0.9491
Starting epoch 8/30
Training Loss: 0.1245, Accuracy: 0.9526
Starting epoch 9/30
Training Loss: 0.1115, Accuracy: 0.9571
Starting epoch 10/30
Training Loss: 0.1042, Accuracy: 0.9597
Starting epoch 11/30
Training Loss: 0.0927, Accuracy: 0.9650
Starting epoch 12/30
Training Loss: 0.0854, Accuracy: 0.9674
Starting epoch 13/30
Training Loss: 0.0789, Accuracy: 0.9708
Starting epoch 14/30
Training Loss: 0.0716, Accuracy: 0.9731
Starting epoch 15/30
Training Loss: 0.0724, Accuracy: 0.9739
Starting epoch 16/30
Training Loss: 0.0635, Accuracy: 0.9770
Starting epoch 17/30
Training Loss: 0.0631, Accuracy: 0.9766
Starting epoch 18/30
Training Loss: 0.0597, Accuracy: 0.9773
Starting epoch 19/30
Training Loss: 0.0572, Accuracy: 0.9784
Starting epoch 20/30
Training Loss: 0.0531, Accuracy: 0.9807
Starting epoch 21/30
Training Loss: 0.0534, Accuracy: 0.9808
Starting epoch 22/30
Training Loss: 0.0508, Accuracy: 0.9813
Starting epoch 23/30
Training Loss: 0.0488, Acc

Training Loss: 0.0789, Accuracy: 0.9708
Starting epoch 14/30
Training Loss: 0.0674, Accuracy: 0.9763
Starting epoch 15/30
Training Loss: 0.0664, Accuracy: 0.9765
Starting epoch 16/30
Training Loss: 0.0599, Accuracy: 0.9787
Starting epoch 17/30
Training Loss: 0.0526, Accuracy: 0.9816
Starting epoch 18/30
Training Loss: 0.0465, Accuracy: 0.9830
Starting epoch 19/30
Training Loss: 0.0498, Accuracy: 0.9822
Starting epoch 20/30
Training Loss: 0.0390, Accuracy: 0.9868
Starting epoch 21/30
Training Loss: 0.0447, Accuracy: 0.9833
Starting epoch 22/30
Training Loss: 0.0360, Accuracy: 0.9874
Starting epoch 23/30
Training Loss: 0.0438, Accuracy: 0.9846
Starting epoch 24/30
Training Loss: 0.0334, Accuracy: 0.9880
Starting epoch 25/30
Training Loss: 0.0346, Accuracy: 0.9889
Starting epoch 26/30
Training Loss: 0.0357, Accuracy: 0.9869
Starting epoch 27/30
Training Loss: 0.0280, Accuracy: 0.9906
Starting epoch 28/30
Training Loss: 0.0291, Accuracy: 0.9902
Starting epoch 29/30
Training Loss: 0.0325, A

Training Loss: 0.0428, Accuracy: 0.9846
Starting epoch 20/30
Training Loss: 0.0371, Accuracy: 0.9863
Starting epoch 21/30
Training Loss: 0.0349, Accuracy: 0.9870
Starting epoch 22/30
Training Loss: 0.0310, Accuracy: 0.9885
Starting epoch 23/30
Training Loss: 0.0365, Accuracy: 0.9876
Starting epoch 24/30
Training Loss: 0.0294, Accuracy: 0.9893
Starting epoch 25/30
Training Loss: 0.0294, Accuracy: 0.9893
Starting epoch 26/30
Training Loss: 0.0263, Accuracy: 0.9904
Starting epoch 27/30
Training Loss: 0.0295, Accuracy: 0.9897
Starting epoch 28/30
Training Loss: 0.0274, Accuracy: 0.9909
Starting epoch 29/30
Training Loss: 0.0251, Accuracy: 0.9916
Starting epoch 30/30
Training Loss: 0.0252, Accuracy: 0.9912
Validation Loss: 0.2130, Accuracy: 0.9440
Training complete..
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
All 8 operations synced, thanks for waiting!
Explore the metadata in the

Training Loss: 0.0261, Accuracy: 0.9909
Starting epoch 26/30
Training Loss: 0.0253, Accuracy: 0.9908
Starting epoch 27/30
Training Loss: 0.0225, Accuracy: 0.9917
Starting epoch 28/30
Training Loss: 0.0241, Accuracy: 0.9914
Starting epoch 29/30
Training Loss: 0.0232, Accuracy: 0.9918
Starting epoch 30/30
Training Loss: 0.0215, Accuracy: 0.9931
Validation Loss: 0.2435, Accuracy: 0.9423
Training complete..
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
All 8 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-640/metadata


#### Hyperparameters and debugging


In [10]:
from collections import defaultdict #neptune_run = defaultdict(list) #
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 1, #30/40
'adam_lr' : 3e-5,
'adam_w' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamw":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_w',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = defaultdict(list) #neptune.init_run() 
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
#neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{params['id']}"
save_history(net,params['network_name'], case_string)


Starting training..
Starting epoch 1/1
Training Loss: 0.4578, Accuracy: 0.8145
Validation Loss: 0.2948, Accuracy: 0.8922
Training complete..


In [8]:
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

/tmp/ipykernel_2702673/459653350.py:28: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  neptune_run = neptune.init_run()


https://app.neptune.ai/aielte/CovidGAN/e/COV1-439
Starting training..
Starting epoch 1/30
Training Loss: 0.4652, Accuracy: 0.8150
Validation Loss: 0.2165, Accuracy: 0.9225
Starting epoch 2/30
Training Loss: 0.2678, Accuracy: 0.8966
Validation Loss: 0.1711, Accuracy: 0.9405
Starting epoch 3/30
Training Loss: 0.2131, Accuracy: 0.9172
Validation Loss: 0.1530, Accuracy: 0.9428
Starting epoch 4/30
Training Loss: 0.1701, Accuracy: 0.9342
Validation Loss: 0.1529, Accuracy: 0.9458
Starting epoch 5/30
Training Loss: 0.1419, Accuracy: 0.9454
Validation Loss: 0.1699, Accuracy: 0.9437
Starting epoch 6/30
Training Loss: 0.0960, Accuracy: 0.9651
Validation Loss: 0.1512, Accuracy: 0.9499
Starting epoch 7/30
Training Loss: 0.0804, Accuracy: 0.9708
Validation Loss: 0.1469, Accuracy: 0.9496
Starting epoch 8/30
Training Loss: 0.0763, Accuracy: 0.9735
Validation Loss: 0.1538, Accuracy: 0.9508
Starting epoch 9/30
Training Loss: 0.0687, Accuracy: 0.9755
Validation Loss: 0.1520, Accuracy: 0.9517
Starting epo

### Longer runs


In [14]:
#1
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#2
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 16,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#3
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 16,
'epochs' : 30, #30/40
'adam_lr' : 12e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#4
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adamW",
'batch_size' : 16,
'epochs' : 30, #30/40
'adam_lr' : 12e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

# 1_None_True_resnet
#5
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#6
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 16,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#7
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 16,
'epochs' : 30, #30/40
'adam_lr' : 12e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#8
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adamW",
'batch_size' : 16,
'epochs' : 30, #30/40
'adam_lr' : 12e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

https://app.neptune.ai/aielte/CovidGAN/e/COV1-641
Starting training..
Starting epoch 1/30
Training Loss: 0.3752, Accuracy: 0.8485
Validation Loss: 0.2009, Accuracy: 0.9272
Starting epoch 2/30
Training Loss: 0.2234, Accuracy: 0.9137
Validation Loss: 0.2201, Accuracy: 0.9172
Starting epoch 3/30
Training Loss: 0.1706, Accuracy: 0.9319
Validation Loss: 0.1631, Accuracy: 0.9446
Starting epoch 4/30
Training Loss: 0.1404, Accuracy: 0.9459
Validation Loss: 0.2096, Accuracy: 0.9178
Starting epoch 5/30
Training Loss: 0.1029, Accuracy: 0.9605
Validation Loss: 0.1783, Accuracy: 0.9369
Starting epoch 6/30
Training Loss: 0.0458, Accuracy: 0.9856
Validation Loss: 0.1533, Accuracy: 0.9537
Starting epoch 7/30
Training Loss: 0.0233, Accuracy: 0.9936
Validation Loss: 0.1665, Accuracy: 0.9523
Starting epoch 8/30
Training Loss: 0.0138, Accuracy: 0.9966
Validation Loss: 0.1780, Accuracy: 0.9526
Starting epoch 9/30
Training Loss: 0.0085, Accuracy: 0.9985
Validation Loss: 0.2007, Accuracy: 0.9514
Starting epo

Validation Loss: 0.2356, Accuracy: 0.9505
Starting epoch 25/30
Training Loss: 0.0021, Accuracy: 0.9998
Validation Loss: 0.2349, Accuracy: 0.9505
Starting epoch 26/30
Training Loss: 0.0021, Accuracy: 0.9998
Validation Loss: 0.2300, Accuracy: 0.9520
Starting epoch 27/30
Training Loss: 0.0023, Accuracy: 0.9997
Validation Loss: 0.2441, Accuracy: 0.9502
Starting epoch 28/30
Training Loss: 0.0024, Accuracy: 0.9997
Validation Loss: 0.2309, Accuracy: 0.9511
Starting epoch 29/30
Training Loss: 0.0027, Accuracy: 0.9997
Validation Loss: 0.2365, Accuracy: 0.9520
Starting epoch 30/30
Training Loss: 0.0020, Accuracy: 0.9999
Validation Loss: 0.2350, Accuracy: 0.9520
Training complete..
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
All 8 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-642/metadata
https://app.neptune.

Starting epoch 16/30
Training Loss: 0.0016, Accuracy: 0.9998
Validation Loss: 0.2259, Accuracy: 0.9499
Starting epoch 17/30
Training Loss: 0.0012, Accuracy: 0.9999
Validation Loss: 0.2225, Accuracy: 0.9505
Starting epoch 18/30
Training Loss: 0.0014, Accuracy: 0.9999
Validation Loss: 0.2299, Accuracy: 0.9514
Starting epoch 19/30
Training Loss: 0.0014, Accuracy: 0.9999
Validation Loss: 0.2255, Accuracy: 0.9517
Starting epoch 20/30
Training Loss: 0.0013, Accuracy: 0.9999
Validation Loss: 0.2273, Accuracy: 0.9508
Starting epoch 21/30
Training Loss: 0.0013, Accuracy: 0.9999
Validation Loss: 0.2287, Accuracy: 0.9496
Starting epoch 22/30
Training Loss: 0.0015, Accuracy: 0.9997
Validation Loss: 0.2337, Accuracy: 0.9496
Starting epoch 23/30
Training Loss: 0.0024, Accuracy: 0.9998
Validation Loss: 0.2252, Accuracy: 0.9496
Starting epoch 24/30
Training Loss: 0.0012, Accuracy: 0.9999
Validation Loss: 0.2344, Accuracy: 0.9502
Starting epoch 25/30
Training Loss: 0.0013, Accuracy: 0.9999
Validation L

Training Loss: 0.0841, Accuracy: 0.9692
Validation Loss: 0.1388, Accuracy: 0.9534
Starting epoch 8/30
Training Loss: 0.0718, Accuracy: 0.9742
Validation Loss: 0.1493, Accuracy: 0.9505
Starting epoch 9/30
Training Loss: 0.0574, Accuracy: 0.9798
Validation Loss: 0.1568, Accuracy: 0.9517
Starting epoch 10/30
Training Loss: 0.0472, Accuracy: 0.9835
Validation Loss: 0.1611, Accuracy: 0.9493
Starting epoch 11/30
Training Loss: 0.0351, Accuracy: 0.9888
Validation Loss: 0.1645, Accuracy: 0.9496
Starting epoch 12/30
Training Loss: 0.0321, Accuracy: 0.9902
Validation Loss: 0.1675, Accuracy: 0.9490
Starting epoch 13/30
Training Loss: 0.0317, Accuracy: 0.9895
Validation Loss: 0.1703, Accuracy: 0.9534
Starting epoch 14/30
Training Loss: 0.0302, Accuracy: 0.9904
Validation Loss: 0.1671, Accuracy: 0.9534
Starting epoch 15/30
Training Loss: 0.0288, Accuracy: 0.9906
Validation Loss: 0.1700, Accuracy: 0.9529
Starting epoch 16/30
Training Loss: 0.0280, Accuracy: 0.9914
Validation Loss: 0.1676, Accuracy: 

Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
All 8 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-647/metadata
https://app.neptune.ai/aielte/CovidGAN/e/COV1-648
Starting training..
Starting epoch 1/30
Training Loss: 0.3938, Accuracy: 0.8477
Validation Loss: 0.2313, Accuracy: 0.9090
Starting epoch 2/30
Training Loss: 0.2711, Accuracy: 0.8969
Validation Loss: 0.1918, Accuracy: 0.9396
Starting epoch 3/30
Training Loss: 0.2320, Accuracy: 0.9132
Validation Loss: 0.1587, Accuracy: 0.9423
Starting epoch 4/30
Training Loss: 0.2030, Accuracy: 0.9239
Validation Loss: 0.1705, Accuracy: 0.9367
Starting epoch 5/30
Training Loss: 0.1788, Accuracy: 0.9316
Validation Loss: 0.1494, Accuracy: 0.9452
Starting epoch 6/30
Training Loss: 0.1089, Accuracy: 0.9588
Validation Loss: 0.1250, Accuracy: 0.9593
Starting epoch 7/30
Training Loss: 0.0886, Accuracy: 0.9673
Validation Lo

In [16]:
#1
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adamW",
'batch_size' : 16,
'epochs' : 30, #30/40
'adam_lr' : 16e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#2
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "resnet", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adamW",
'batch_size' : 16,
'epochs' : 30, #30/40
'adam_lr' : 16e-5,
'adam_weight_decay' :0.1,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
if params['optimizer']=="adam":
    optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
elif params['optimizer']=="adamW":
    optimizer = torch.optim.AdamW(network.parameters(), lr=params['adam_lr'], weight_decay=params.get('adam_weight_decay',0.1))
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

https://app.neptune.ai/aielte/CovidGAN/e/COV1-651
Starting training..
Starting epoch 1/30
Training Loss: 0.4062, Accuracy: 0.8397
Validation Loss: 0.2074, Accuracy: 0.9196
Starting epoch 2/30
Training Loss: 0.2689, Accuracy: 0.8971
Validation Loss: 0.2114, Accuracy: 0.9269
Starting epoch 3/30
Training Loss: 0.2225, Accuracy: 0.9158
Validation Loss: 0.2328, Accuracy: 0.9154
Starting epoch 4/30
Training Loss: 0.1916, Accuracy: 0.9280
Validation Loss: 0.1546, Accuracy: 0.9437
Starting epoch 5/30
Training Loss: 0.1608, Accuracy: 0.9398
Validation Loss: 0.1829, Accuracy: 0.9299
Starting epoch 6/30
Training Loss: 0.0783, Accuracy: 0.9731
Validation Loss: 0.1272, Accuracy: 0.9561
Starting epoch 7/30
Training Loss: 0.0459, Accuracy: 0.9848
Validation Loss: 0.1384, Accuracy: 0.9570
Starting epoch 8/30
Training Loss: 0.0260, Accuracy: 0.9919
Validation Loss: 0.1639, Accuracy: 0.9496
Starting epoch 9/30
Training Loss: 0.0140, Accuracy: 0.9961
Validation Loss: 0.1900, Accuracy: 0.9517
Starting epo

Validation Loss: 0.1591, Accuracy: 0.9534
Starting epoch 25/30
Training Loss: 0.0283, Accuracy: 0.9917
Validation Loss: 0.1575, Accuracy: 0.9555
Starting epoch 26/30
Training Loss: 0.0285, Accuracy: 0.9909
Validation Loss: 0.1593, Accuracy: 0.9526
Starting epoch 27/30
Training Loss: 0.0284, Accuracy: 0.9914
Validation Loss: 0.1563, Accuracy: 0.9552
Starting epoch 28/30
Training Loss: 0.0290, Accuracy: 0.9912
Validation Loss: 0.1618, Accuracy: 0.9537
Starting epoch 29/30
Training Loss: 0.0278, Accuracy: 0.9919
Validation Loss: 0.1595, Accuracy: 0.9549
Starting epoch 30/30
Training Loss: 0.0275, Accuracy: 0.9915
Validation Loss: 0.1590, Accuracy: 0.9561
Training complete..
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
All 8 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-652/metadata
